# Introduction to NLP Fundamentals in Tensorflow

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
2.14.0-dev20230730


In [2]:
import os

if not os.path.exists("helper_functions.py"):
  !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
  print("[INFO] 'helper_functions.py' already exists, skipping download.")

# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

[INFO] 'helper_functions.py' already exists, skipping download.


We will be using RNN to train textual information

**Basic Structure of RNN**
* Input layer
* text_vectorizer(input)
* embedding(x)
* LSTM -> activation = tanh
* output_Dense -> activation = sigmoid

### Get the text dataset

In [3]:
if not os.path.exists("nlp_getting_started.zip"):
    !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
    unzip_data("nlp_getting_started.zip")
else:
  print("[INFO] already exists, skipping download.")

[INFO] already exists, skipping download.


### Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [5]:
train_df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [6]:
#Shuffle to train data
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head(10)

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
5559,7934,rainstorm,NaN,@Calum5SOS you look like you got caught in a r...,0
1765,2538,collision,NaN,my favorite lady came to our volunteer meeting...,1
1817,2611,crashed,NaN,@brianroemmele UX fail of EMV - people want to...,1
6810,9756,tragedy,"Los Angeles, CA",Can't find my ariana grande shirt this is a f...,0
4398,6254,hijacking,"Athens,Greece",The Murderous Story Of AmericaÛªs First Hijac...,1


In [7]:
# test_data
test_df.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


In [8]:
#How many examples in each class
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
# visualise some random training examples
import random
random_index = random.randint(0,len(train_df)-5)
print(random_index)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  index,text,target = row
  print(f"Target: {target}, {'(real disaster)' if target > 0 else '(not a real disaster)'}")
  print(f"Text:\n{text}\n")
  print("---")

3259
Target: 1, (real disaster)
Text:
Severe thunderstorm warning remains for #Cochrane. @cityofcalgary has enacted municipal emergency plan after today's storm. #abstorm

---
Target: 0, (not a real disaster)
Text:
Just had a heart attack because I thought my goat was dead. ???? don't worry Rocket is okay. ??

---
Target: 1, (real disaster)
Text:
FreeBesieged: .MartinMJ22 YouGov Which '#Tory landslide' ... you can't POSSIBLY mean the wafer-thin majority of #GÛ_ http://t.co/2q3fuEReY5

---
Target: 1, (real disaster)
Text:
Video:  Fire burns two apartment buildings and blows up car in Manchester http://t.co/5BGcw3EzB5

---
Target: 1, (real disaster)
Text:
Men escape car engulfed in flames in Parley's Canyon crews investigating cause - http://t.co/tFan6qq2Ys http://t.co/rAkwWritPo

---


In [10]:
len(train_df_shuffled)

7613

In [11]:
#Splitting the data to create validation -> (train + validation)
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [12]:
#Check length
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check few sample
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

1. tokenization -> can get too big as the number of text sample increases
2. Embedding -> richer representation of relationship be between tokens, deeper the realtion between words, more the number/embedding for that token

In [14]:
import tensorflow as tf

# Use the default TextVectorization variables
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                                    standardize="lower_and_strip_punctuation", # how to process text
                                                    split="whitespace", # how to split tokens
                                                    ngrams=None, # create groups of n-words?
                                                    output_mode="int", # how to map tokens to numbers
                                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                    output_mode="int",
                                                    output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [19]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
? This Weekend: Stockholm Sweden - Aug 8 at Copperfields http://t.co/6un7xC9Sve      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  19, 1238, 8067, 3363,  795,  589,   17,    1,    1,    0,    0,
           0,    0,    0,    0]])>

In [20]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [21]:
tf.random.set_seed(42)

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [22]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
OH. #TeamHennessy #NJ Obliteration  @tprimo24 ROUND 1 Happy Birthday @djeddygnj Colombian FestivalÛ_ https://t.co/mRv54fiDfn

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.020172  , -0.02847883,  0.04023199, ..., -0.04029989,
         -0.03703539, -0.00482714],
        [-0.01231217,  0.03816457,  0.03293427, ...,  0.02968596,
          0.02022697,  0.03278508],
        [-0.03631087,  0.0317595 ,  0.01052377, ...,  0.00781127,
          0.04762465,  0.01803061],
        ...,
        [-0.01102264, -0.00198046, -0.01452967, ...,  0.01859972,
         -0.04886992, -0.0298776 ],
        [ 0.02283308,  0.01946468, -0.03277663, ...,  0.00894006,
          0.01734594,  0.02719237],
        [ 0.02283308,  0.01946468, -0.03277663, ...,  0.00894006,
          0.01734594,  0.02719237]]], dtype=float32)>

In [23]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.020172  , -0.02847883,  0.04023199,  0.04728966,  0.00186285,
        0.04499518,  0.04994962,  0.01597241,  0.01229657,  0.01284934,
        0.04454099, -0.02522776, -0.04172603, -0.03823409, -0.04884083,
        0.00206297,  0.00718277, -0.00372468,  0.04055433, -0.03480301,
       -0.04730551, -0.04665476, -0.04367707, -0.00369298, -0.00073723,
        0.01998222,  0.03300932, -0.0047936 ,  0.02777244,  0.00257748,
        0.02543373, -0.04042919, -0.04620775, -0.01492273, -0.00447705,
        0.03758103, -0.02014437, -0.04566283, -0.00324927, -0.01271741,
        0.00503214, -0.01612542,  0.01808793, -0.03852513, -0.03048076,
       -0.02755363,  0.03771399,  0.03776829,  0.0464023 ,  0.01933909,
       -0.03063407,  0.04747662,  0.04012598, -0.00700953,  0.04503801,
        0.01490257, -0.00628858, -0.03376269, -0.03966572, -0.04844163,
       -0.04529866,  0.03773271,  0.04353518, -0.0225172 ,  0.0496343 ,
       -0.019229

### Modelling a text dataset
More specifically, we'll be building the following:

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neural network (dense model)
* Model 2: LSTM model
* Model 3: GRU model
* Model 4: Bidirectional-LSTM model
* Model 5: 1D Convolutional Neural Network
* Model 6: TensorFlow Hub Pretrained Feature Extractor
* Model 7: Same as model 6 with 10% of training data



In [24]:
#model 0
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [25]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [26]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
from helper_functions import calculate_results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 1: A simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

# Build model with the Functional API
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [29]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230815-082808


Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.6091 - accuracy: 0.6946 - val_loss: 0.5357 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.4412 - accuracy: 0.8190 - val_loss: 0.4690 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3465 - accuracy: 0.8616 - val_loss: 0.4587 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2850 - accuracy: 0.8923 - val_loss: 0.4638 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2380 - accuracy: 0.9124 - val_loss: 0.4764 - val_accuracy: 0.7874


In [30]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 1ms/step - loss: 0.4764 - accuracy: 0.7874


[0.476399302482605, 0.787401556968689]

#### Recurrent Neural Network
* RNN's are useful for sequence data
* The premise of a recurrent neural network is to use the representation of previous input to aid the representation of a later input

### Model 2 : LSTM -> Long short term memory

Input(text) -> Toeknize -> Embedding -> Layers(RNN/Dense) -> Output(label probability)

In [31]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)

model_2_embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = tf.keras.layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)

print(x.shape)

x = tf.keras.layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x) # return vector for whole sequence

print(x.shape)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs)
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_2.summary()

(None, 15, 128)


(None, 64)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 lstm_1 (LSTM)               (None, 15, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                  

In [32]:
# Fit model
model_2_history = model_2.fit(train_sentences, train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, 
                                                                     "LSTM")])

Saving TensorBoard log files to: model_logs/LSTM/20230815-082816
Epoch 1/5


215/215 [==============================] - 7s 21ms/step - loss: 0.5066 - accuracy: 0.7526 - val_loss: 0.4557 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3253 - accuracy: 0.8716 - val_loss: 0.5068 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2275 - accuracy: 0.9133 - val_loss: 0.5487 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1692 - accuracy: 0.9388 - val_loss: 0.5897 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1158 - accuracy: 0.9550 - val_loss: 0.9578 - val_accuracy: 0.7664


In [33]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.01341139],
       [0.63916904],
       [0.9996854 ],
       [0.05075767],
       [0.00136014],
       [0.9998703 ],
       [0.6901335 ],
       [0.9998673 ],
       [0.9998171 ],
       [0.26813403]], dtype=float32)

In [34]:
# Convert preds to laels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [35]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [36]:
model_2_results = calculate_results(val_labels, model_2_preds)
model_2_results

{'accuracy': 76.64041994750657,
 'precision': 0.7684333697179313,
 'recall': 0.7664041994750657,
 'f1': 0.7640154753782149}

#### Model 3: GRU

1. Gated recurrent unit is effective and popular
2. GRU cell has similar features to LSTM cell but has less paramenter

In [37]:
# Build GRU RNN
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.GRU(64, return_sequences=True)(x)
# x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.GRU(64)(x)
# x = tf.keras.layers.Dense(64,activation="relu")(x)
outputs = tf.keras.layers.Dense(1,activation="sigmoid")(x)

model_3 = tf.keras.Model(inputs,outputs)

model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences,val_labels))

Epoch 1/5


215/215 [==============================] - 5s 18ms/step - loss: 0.2395 - accuracy: 0.8986 - val_loss: 0.5282 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.1608 - accuracy: 0.9423 - val_loss: 0.5954 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1285 - accuracy: 0.9531 - val_loss: 0.7467 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.1095 - accuracy: 0.9604 - val_loss: 0.7576 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0923 - accuracy: 0.9669 - val_loss: 0.7341 - val_accuracy: 0.7900


In [38]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[5.8749992e-02],
       [8.2415801e-01],
       [9.9943858e-01],
       [4.7681525e-02],
       [8.5235102e-04],
       [9.8981863e-01],
       [5.8692926e-01],
       [9.9971873e-01],
       [9.9935442e-01],
       [6.5132934e-01]], dtype=float32)

In [39]:
# Convert model 3 pred_probs to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [40]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 79.00262467191601,
 'precision': 0.7918692779467839,
 'recall': 0.7900262467191601,
 'f1': 0.788238745723847}

#### Model4 - Bidirectional RNN

In [41]:
# Trying out on my own
text_vectorizer_exp = tf.keras.layers.TextVectorization(max_tokens=10000,
                                      output_sequence_length=15)
text_vectorizer_exp.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer_exp(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4_exp = tf.keras.Model(inputs,outputs)
model_4_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_4_history_exp = model_4_exp.fit(train_sentences,
                                      train_labels,
                                      validation_data=(val_sentences,val_labels),
                                      epochs=5)

Epoch 1/5
215/215 [==============================] - 7s 22ms/step - loss: 0.5042 - accuracy: 0.7500 - val_loss: 0.4532 - val_accuracy: 0.7874
Epoch 2/5
215/215 [==============================] - 1s 7ms/step - loss: 0.3128 - accuracy: 0.8734 - val_loss: 0.5126 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2097 - accuracy: 0.9197 - val_loss: 0.5580 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.1433 - accuracy: 0.9521 - val_loss: 0.6834 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0954 - accuracy: 0.9677 - val_loss: 0.7897 - val_accuracy: 0.7585


Normal RNN's go from left to right

Bi-directional go from right to left as well as left to right

In [42]:
# model 4
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                    output_sequence_length=15)
text_vectorizer.adapt(train_sentences)

inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = tf.keras.layers.Embedding(input_dim=10000,
                              output_dim=128,
                              input_length=15)(x)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_4 = tf.keras.Model(inputs,outputs)

model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_4_history = model_4.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels))

Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.5091 - accuracy: 0.7430 - val_loss: 0.4622 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3128 - accuracy: 0.8720 - val_loss: 0.5172 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2112 - accuracy: 0.9191 - val_loss: 0.5642 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1449 - accuracy: 0.9511 - val_loss: 0.6561 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1039 - accuracy: 0.9628 - val_loss: 0.6965 - val_accuracy: 0.7625


In [43]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 1ms/step


array([[0.0416243 ],
       [0.7163756 ],
       [0.9992931 ],
       [0.12213274],
       [0.0041174 ],
       [0.99123585],
       [0.8382135 ],
       [0.99947184],
       [0.99936336],
       [0.15541679]], dtype=float32)

In [44]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
model_4_results = calculate_results(val_labels,model_4_preds)
model_4_results

{'accuracy': 76.24671916010499,
 'precision': 0.7625326572242482,
 'recall': 0.7624671916010499,
 'f1': 0.7612168110136286}

#### 1D CNN

In [46]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = tf.keras.layers.Conv1D(32,
                                 5,
                                 activation="relu",
                                 padding="valid")
conv_1d_output = conv_1d(embedding_test)

max_pool = tf.keras.layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [47]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04114373, -0.03656233, -0.01222943, ..., -0.05614972,
         -0.05432083,  0.00050777],
        [ 0.00626119, -0.01586794,  0.01605864, ...,  0.00363537,
         -0.00085598,  0.0574589 ],
        [-0.02066967,  0.00708318, -0.02963221, ..., -0.01524623,
         -0.01601685,  0.02936716],
        ...,
        [ 0.00772676,  0.02109918, -0.0082626 , ..., -0.00559928,
          0.00864187,  0.02604301],
        [ 0.00772676,  0.02109918, -0.0082626 , ..., -0.00559928,
          0.00864187,  0.02604301],
        [ 0.00772676,  0.02109918, -0.0082626 , ..., -0.00559928,
          0.00864187,  0.02604301]]], dtype=float32)>

In [48]:
conv_1d_output

<tf.Tensor: shape=(1, 11, 32), dtype=float32, numpy=
array([[[0.00000000e+00, 5.51878065e-02, 9.26857907e-03, 1.44575862e-02,
         0.00000000e+00, 6.66530430e-02, 1.65071059e-02, 2.82363817e-02,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.88682103e-02,
         1.46425003e-02, 2.50137299e-02, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 3.45092081e-02, 4.89880741e-02,
         3.87286060e-02, 0.00000000e+00, 0.00000000e+00, 1.26618713e-01,
         4.64972481e-03, 2.09586509e-03, 0.00000000e+00, 1.85611136e-02,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.61853861e-02, 2.25222930e-02, 4.82707098e-03, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 1.59236249e-02, 0.00000000e+00,
         1.05586134e-01, 0.00000000e+00, 0.00000000e+00, 5.41576818e-02,
         0.00000000e+00, 9.83871669e-02, 7.28666037e-03, 0.00000000e+00,
         1.17224278e-02, 0.00000000e+00, 2.38685869e-03, 2.72989254e-0

In [49]:
max_pool_output

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[0.0667422 , 0.05518781, 0.00926858, 0.02502883, 0.03327907,
        0.06665304, 0.04768277, 0.02823638, 0.10558613, 0.03249618,
        0.02906286, 0.08625288, 0.04933174, 0.09838717, 0.02599783,
        0.0004406 , 0.01628369, 0.01738921, 0.03668899, 0.04898807,
        0.03872861, 0.        , 0.08217077, 0.12661871, 0.05410383,
        0.07303643, 0.07894074, 0.11399034, 0.        , 0.04690762,
        0.06161265, 0.03076662]], dtype=float32)>

In [50]:
# Create 1D CNN layer to model sequences
inputs = tf.keras.layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.Conv1D(filters=64,
                           kernel_size=5,
                           padding="valid",
                           activation="relu")(x)
x = tf.keras.layers.GlobalMaxPool1D()(x)
outputs = tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)(x)

model_5 = tf.keras.Model(inputs,outputs)

model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5_history = model_5.fit(train_sentences,train_labels,
                              validation_data=(val_sentences,val_labels),
                              epochs=5)

Epoch 1/5


215/215 [==============================] - 5s 18ms/step - loss: 0.1545 - accuracy: 0.9457 - val_loss: 0.7496 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1023 - accuracy: 0.9615 - val_loss: 0.8862 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.0804 - accuracy: 0.9685 - val_loss: 0.9640 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 1s 3ms/step - loss: 0.0696 - accuracy: 0.9729 - val_loss: 1.0453 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.0606 - accuracy: 0.9753 - val_loss: 1.1089 - val_accuracy: 0.7703


In [51]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 913us/step


array([[2.1980181e-01],
       [6.2039989e-01],
       [9.9994481e-01],
       [4.5346811e-02],
       [3.9272891e-07],
       [9.9793768e-01],
       [9.7992134e-01],
       [9.9986410e-01],
       [9.9999857e-01],
       [7.0635623e-01]], dtype=float32)

In [52]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [53]:
model_5_results = calculate_results(val_labels,model_5_preds)
model_5_results

{'accuracy': 77.03412073490814,
 'precision': 0.7708102089179978,
 'recall': 0.7703412073490814,
 'f1': 0.7689180260110319}

In [54]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

#### Model 6 - Tensorflow Hub pretrained models
USE-extractor - universal sentence encoder (USE)

In [55]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed([sample_sentence,
                      "When you can the unvidersal sentence encider on a sentence, it turns into into numbers"])
print(embed_sample[0][:50])

tf.Tensor(
[-0.01157027  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680985  0.05589838 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105888  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209305  0.0267898   0.03011008  0.00321069
 -0.00337971 -0.04787356  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666384  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871508  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758105  0.03327729], shape=(50,), dtype=float32)


In [56]:
embed_sample

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.01157027,  0.0248591 ,  0.02878048, ..., -0.00186125,
         0.02315824, -0.01485021],
       [ 0.06090042, -0.08857404,  0.01838204, ..., -0.00653699,
         0.03106263,  0.03701008]], dtype=float32)>

Use the above USE"" embedding layer for a normal type of model

In [57]:
# Create a keras layer using the USE"" pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer(hub.load("https://tfhub.dev/google/universal-sentence-encoder/4"),
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [58]:
sentence_encoder_layer

In [59]:
# Create  sequential layer
model_6 = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(1,activation="sigmoid")
])

model_6.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_6_history = model_6.fit(train_sentences,train_labels,
                               validation_data=(val_sentences,val_labels),
                               epochs=5)

Epoch 1/5
215/215 [==============================] - 2s 6ms/step - loss: 0.6484 - accuracy: 0.7313 - val_loss: 0.6127 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5812 - accuracy: 0.7917 - val_loss: 0.5630 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5382 - accuracy: 0.7957 - val_loss: 0.5311 - val_accuracy: 0.7848
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.5095 - accuracy: 0.7984 - val_loss: 0.5098 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4894 - accuracy: 0.8003 - val_loss: 0.4953 - val_accuracy: 0.7887


In [60]:
model_6_pred_probs=model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[0.3678494 ],
       [0.6996462 ],
       [0.85615575],
       [0.337918  ],
       [0.63912416],
       [0.73496485],
       [0.8274349 ],
       [0.8328631 ],
       [0.749073  ],
       [0.18539421]], dtype=float32)

In [61]:
model_6_preds=tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_6_results = calculate_results(val_labels,model_6_preds)
model_6_results

{'accuracy': 78.87139107611549,
 'precision': 0.7894940548343282,
 'recall': 0.7887139107611548,
 'f1': 0.7874045839301492}

In [63]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [88]:
# Challenge, beat the baseline
model_6_exp = tf.keras.models.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(64,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid)
])
model_6_exp.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
model_6_exp_history = model_6_exp.fit(train_sentences,train_labels,
                                      epochs=5,
                                      validation_data=(val_sentences,val_labels))

model_6_exp_pred_probs = model_6_exp.predict(val_sentences)
print(model_6_exp_pred_probs[:10])

model_6_exp_preds = tf.squeeze(tf.round(model_6_exp_pred_probs))
print(model_6_exp_preds[:10])

model_6_exp_results = calculate_results(val_labels, model_6_exp_preds)
print(model_6_exp_results)

Epoch 1/5


215/215 [==============================] - 2s 6ms/step - loss: 0.4860 - accuracy: 0.7895 - val_loss: 0.4331 - val_accuracy: 0.8018
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4021 - accuracy: 0.8209 - val_loss: 0.4177 - val_accuracy: 0.8241
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3799 - accuracy: 0.8354 - val_loss: 0.4153 - val_accuracy: 0.8228
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3586 - accuracy: 0.8495 - val_loss: 0.4143 - val_accuracy: 0.8202
Epoch 5/5
24/24 [==============================] - 0s 4ms/step
[[0.13529278]
 [0.8153213 ]
 [0.9954295 ]
 [0.21450908]
 [0.6610022 ]
 [0.8446863 ]
 [0.9911315 ]
 [0.9947973 ]
 [0.9610625 ]
 [0.06273486]]
tf.Tensor([0. 1. 1. 0. 1. 1. 1. 1. 1. 0.], shape=(10,), dtype=float32)
{'accuracy': 81.88976377952756, 'precision': 0.8225023097234344, 'recall': 0.8188976377952756, 'f1': 0.8170457056303013}
